In [1]:
import pyomo.environ as pyo 
from pyomo.environ import *
from pyomo.opt import SolverFactory
import math 
import pandas as pd
import os
os.chdir('C:\\Users\\omkarp\\Downloads\\Optimize\\NonlinearOptimization\\')

# Project: Planning of electricity production and transmission

1. Formulate a model minimizing the cost of production of active power

**Parameters for the model:**

- Nodes (k,l) -ranging from 1 to 11 over transmission grid network
- Shunt admittance on edges:   $b_{kl}$ and $g_{kl}$ between nodes
- Generator Max Capacity (pu):  $G_{ki}$ with i for Generator number ranging from 1 to 9, and k for node location
- Generator Energy production cost (SEK/pu):  $GEcost_{ki}$ with i for Gen.no. ranging from 1 to 9, and k for node location
- Consumer Demand for Active Power: $C_{li}$ with i for Consumer number ranging from 1 to 7, and l for node location


## Get data

In [2]:
shunt_data = pd.read_excel('Project.xlsx', sheet_name='shunt')
shunt_data[['k', 'l']] = shunt_data['Edge(kl)'].str.split('_', expand=True)
shunt_data['k'] = shunt_data['k'].astype(int)
shunt_data['l'] = shunt_data['l'].astype(int)
shunt_data.head()

,Edge(kl),bkl,gkl,k,l
0,1_2,-20.1,4.12,1,2
1,1_11,-22.3,5.67,1,11
2,2_3,-16.8,2.41,2,3
3,2_11,-17.2,2.78,2,11
4,3_4,-11.7,1.98,3,4


In [3]:
generator_data = pd.read_excel('Project.xlsx', sheet_name='generator')
consumer_data = pd.read_excel('Project.xlsx', sheet_name='consumer')
generator_data

,Generator,node,MaxCapacity,Energy_prod_cost
0,G1,2,0.02,175
1,G2,2,0.15,100
2,G3,2,0.08,150
3,G4,3,0.07,150
4,G5,4,0.04,300
5,G6,5,0.17,350
6,G7,7,0.17,400
7,G8,9,0.26,300
8,G9,9,0.05,200


In [4]:
print([generator_data.MaxCapacity[i] for i in generator_data.index])
generator_data.info()

[0.02, 0.15, 0.08, 0.07, 0.04, 0.17, 0.17, 0.26, 0.05]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Generator         9 non-null      object 
 1   node              9 non-null      int64  
 2   MaxCapacity       9 non-null      float64
 3   Energy_prod_cost  9 non-null      int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 420.0+ bytes


In [5]:
# powerplant = pd.read_excel('Project.xlsx', sheet_name='PowerPlant') # not used
consumer_data

,Consumer,node,Demand_active_power
0,C1,1,0.10
1,C2,4,0.19
2,C3,6,0.11
3,C4,8,0.09
4,C5,9,0.21
5,C6,10,0.05
6,C7,11,0.04


## **MappIng nodes - edges to generators and consumers and power-plants for the model:**

In [6]:
node_to_edges = {}
node_to_generators = {}
node_to_consumers = {}

connections = {
    1: [2, 11],
    2: [1, 3, 11],
    3: [2, 4, 9],
    4: [3, 5],
    5: [4, 6, 8],
    6: [5, 7],
    7: [6, 8, 9],
    8: [5, 7, 9],
    9: [10, 3, 8, 7],
    10: [9, 11],
    11: [1, 2, 10]
}
for node, connected_nodes in connections.items():
    node_to_edges[node] = connected_nodes
    
node_to_consumers[1] = ['C1']                 # Node 1
node_to_generators[2] = ['G1', 'G2', 'G3']    # Node 2
node_to_generators[3] = ['G4']                # Node 3
node_to_generators[4] = ['G5']                # Node 4
node_to_consumers[4] = ['C2']                 # Node 4
node_to_generators[5] = ['G6']                # Node 5
node_to_consumers[6] = ['C3']                 # Node 6
node_to_generators[7] = ['G7']                # Node 7
node_to_consumers[8] = ['C4']                 # Node 8
node_to_consumers[9] = ['C5']                 # Node 9
node_to_generators[9] = ['G8', 'G9']          # Node 9
node_to_consumers[10] = ['C6']                # Node 10
node_to_consumers[11] = ['C7']                # Node 11

In [7]:
node = 9
if node in node_to_edges:
    edges = node_to_edges[node]
    print(f"Edges for Node {node}: {edges}")
if node in node_to_generators:
    generators = node_to_generators[node]
    print(f"Generators for Node {node}: {generators}")
if node in node_to_consumers:
    consumers = node_to_consumers[node]
    print(f"Consumers for Node {node}: {consumers}")

Edges for Node 9: [10, 3, 8, 7]
Generators for Node 9: ['G8', 'G9']
Consumers for Node 9: ['C5']


In [8]:
model = pyo.ConcreteModel()

### Setting model parameters

In [9]:
# Sets
node_indices = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
model.NODE = pyo.Set(initialize=node_indices)

generator_indices = {name: idx for idx, name in enumerate(generator_data['Generator'])} # initialize as integer indices
model.GENERATOR = pyo.Set(initialize=generator_indices.values())

consumer_indices = {name: idx for idx, name in enumerate(['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7'])}
model.CONSUMER = pyo.Set(initialize=consumer_indices.values())

In [10]:
# Parameters: Generator
max_capacity_values = {}
energy_cost_values = {}
generator_location = {}

for generator in generator_data.iterrows():
    generator_name = generator[1]['Generator']
    generator_idx = generator_indices[generator_name]  # integer index
    node_location = generator[1]['node']
    max_capacity = generator[1]['MaxCapacity']
    max_capacity_values[generator_idx] = max_capacity
    generator_location[generator_idx] = node_location
    
for generator in generator_data.iterrows():
    generator_name = generator[1]['Generator']
    generator_idx = generator_indices[generator_name] 
    energy_cost = generator[1]['Energy_prod_cost']
    energy_cost_values[generator_idx] = energy_cost
    
model.max_capacity = pyo.Param(model.GENERATOR, initialize=max_capacity_values)
model.cost = pyo.Param(model.GENERATOR, initialize=energy_cost_values)

generator_location_values = {generator_idx: node_location for generator_idx, node_location in generator_location.items()}
model.generator_location = pyo.Param(model.GENERATOR, initialize=generator_location_values)

for generator in model.GENERATOR:
    max_capacity_value = model.max_capacity[generator]
    print(f'Max Capacity for Generator {generator+1} = {max_capacity_value} pu')

Max Capacity for Generator 1 = 0.02 pu
Max Capacity for Generator 2 = 0.15 pu
Max Capacity for Generator 3 = 0.08 pu
Max Capacity for Generator 4 = 0.07 pu
Max Capacity for Generator 5 = 0.04 pu
Max Capacity for Generator 6 = 0.17 pu
Max Capacity for Generator 7 = 0.17 pu
Max Capacity for Generator 8 = 0.26 pu
Max Capacity for Generator 9 = 0.05 pu


In [11]:
# Parameters: Consumers
demand_values = {}
consumer_location = {}

for consumer in consumer_data.iterrows():
    consumer_name = consumer[1]['Consumer']
    node_location = consumer[1]['node']
    demand = consumer[1]['Demand_active_power']
    consumer_idx = consumer_indices[consumer_name]
    demand_values[consumer_idx] = demand
    consumer_location[consumer_idx] = node_location
    
model.demand = pyo.Param(model.CONSUMER, initialize=demand_values, default=0.0)
model.consumer_location = pyo.Param(model.CONSUMER, initialize=consumer_location)

for consumer in model.CONSUMER:
    demand_value = model.demand[consumer]
    print(f'Demand active power for Consumer {consumer+1} = {demand_value} pu')

Demand active power for Consumer 1 = 0.1 pu
Demand active power for Consumer 2 = 0.19 pu
Demand active power for Consumer 3 = 0.11 pu
Demand active power for Consumer 4 = 0.09 pu
Demand active power for Consumer 5 = 0.21 pu
Demand active power for Consumer 6 = 0.05 pu
Demand active power for Consumer 7 = 0.04 pu


In [12]:
# Parameters for Shunt Admittance
bkl_values = {}
gkl_values = {}

for k, connected_nodes in node_to_edges.items():
    for l in connected_nodes:
        if k != l:
            matching_rows = shunt_data[(shunt_data['k'] == k) & (shunt_data['l'] == l)]
            if not matching_rows.empty:
                gkl = matching_rows['gkl'].values[0]
                bkl = matching_rows['bkl'].values[0]
                gkl_values[(k, l)] = gkl
                gkl_values[(l, k)] = gkl
                bkl_values[(k, l)] = bkl
                bkl_values[(l, k)] = bkl

model.bkl = Param(model.NODE, model.NODE, initialize=bkl_values)
model.gkl = Param(model.NODE, model.NODE, initialize=gkl_values)

In [13]:
for k, connected_nodes in node_to_edges.items():
    for l in connected_nodes:
        if (k, l) in model.gkl:
            gkl_value = model.gkl[k, l]
            print(f'gkl[{k}, {l}] = {gkl_value}')
            break

gkl[1, 2] = 4.12
gkl[2, 1] = 4.12
gkl[3, 2] = 2.41
gkl[4, 3] = 1.98
gkl[5, 4] = 1.59
gkl[6, 5] = 1.71
gkl[7, 6] = 1.11
gkl[8, 5] = 1.26
gkl[9, 10] = 2.14
gkl[10, 9] = 2.14
gkl[11, 1] = 5.67


## Set variables

**Variables for the model:**

- Voltage Amplitude ($v_{k}$ for each node k) between 0.98 and 1.00
- Voltage angles ($\theta_{k}$ for each node k) between pi and -pi (radians)
- Power flows for active ($p.var_{k}$ for each node k) and for reactive ($q.var_{k}$ for each node k)
- Generated active and reactive power by each generator

In [14]:
# Variables
model.v = pyo.Var(model.NODE, bounds=(0.98, 1.02))  # voltage amplitudes
model.theta = pyo.Var(model.NODE, bounds=(-math.pi, math.pi))  # voltage angle (in radians - math.pi)

# Active and Reactive Power Flow for each edge
model.p_var = pyo.Var(model.NODE, model.NODE, within=pyo.Reals)  # active
model.q_var = pyo.Var(model.NODE, model.NODE, within=pyo.Reals)  # reactive

# Active and Reactive Power Generation by each generator
model.active_power = pyo.Var(model.GENERATOR, within=pyo.NonNegativeReals)
model.reactive_power = pyo.Var(model.GENERATOR, within=pyo.Reals)

## Set Constraints

In [15]:
# Notify constraints for active power generation
def active_power_notify_rule(model, generator):
    if generator in [1, 6, 8, 10, 11]:
        return model.active_power[generator] == 0.0
    elif generator in [2, 3, 4, 5, 7, 9]:
        return model.active_power[generator] == sum(model.active_power[g] for g in model.GENERATOR if model.generator_location[g] == generator)
    else:
        return pyo.Constraint.Skip  # Handle other cases if needed

model.active_power_notify_constraints = pyo.Constraint(model.GENERATOR, rule=active_power_notify_rule)

# Notify constraints for reactive power generation
def reactive_power_notify_rule(model, generator):
    if generator in [1, 6, 8, 10, 11]:
        return model.reactive_power[generator] == 0.0
    elif generator in [2, 3, 4, 5, 7, 9]:
        return model.reactive_power[generator] == sum(model.reactive_power[g] for g in model.GENERATOR if model.generator_location[g] == generator)
    else:
        return pyo.Constraint.Skip  # Handle other cases if needed

model.reactive_power_notify_constraints = pyo.Constraint(model.GENERATOR, rule=reactive_power_notify_rule)

In [16]:
# Bounds for active and reactive power generation by each generator
def active_power_bounds_rule(model, generator):
    max_capacity = model.max_capacity[generator]
    return (0, max_capacity)

model.active_power_bounds = pyo.Constraint(model.GENERATOR, rule=active_power_bounds_rule)

def reactive_power_bounds_rule(model, generator):
    max_capacity = model.max_capacity[generator]
    return (-0.003 * max_capacity, 0.003 * max_capacity)

model.reactive_power_bounds = pyo.Constraint(model.GENERATOR, rule=reactive_power_bounds_rule)

### **Amount of active power flow (p_var) between nodes**

$$\begin{aligned}
p_{kℓ} = (v_k)^2 * g_{kℓ} − v_k * v_ℓ * g_{kℓ} * cos(θ_k − θ_ℓ) − v_k * v_ℓ * b_{kℓ} * sin(θ_k − θ_ℓ)
\end{aligned}$$

In [17]:
def active_power_flow_rule(model, k, l):
    if (k, l) in model.gkl:
        p_flow = (model.v[k] ** 2) * model.gkl[k, l] - model.v[k] * model.v[l] * model.gkl[k, l] * pyo.cos(model.theta[k] - model.theta[l]) - model.v[k] * model.v[l] * model.bkl[k, l] * pyo.sin(model.theta[k] - model.theta[l])
        print(f"Edge ({k}, {l}) - Active Power Flow: {p_flow}")
        return model.p_var[k, l] == p_flow
    else:
        return pyo.Constraint.Skip

model.active_power_flow_constraint = pyo.Constraint(model.NODE, model.NODE, rule=active_power_flow_rule)

Edge (1, 2) - Active Power Flow: v[1]**2*4.12 - v[1]*v[2]*4.12*cos(theta[1] - theta[2]) - v[1]*v[2]*-20.1*sin(theta[1] - theta[2])
Edge (1, 11) - Active Power Flow: v[1]**2*5.67 - v[1]*v[11]*5.67*cos(theta[1] - theta[11]) - v[1]*v[11]*-22.3*sin(theta[1] - theta[11])
Edge (2, 1) - Active Power Flow: v[2]**2*4.12 - v[2]*v[1]*4.12*cos(theta[2] - theta[1]) - v[2]*v[1]*-20.1*sin(theta[2] - theta[1])
Edge (2, 3) - Active Power Flow: v[2]**2*2.41 - v[2]*v[3]*2.41*cos(theta[2] - theta[3]) - v[2]*v[3]*-16.8*sin(theta[2] - theta[3])
Edge (2, 11) - Active Power Flow: v[2]**2*2.78 - v[2]*v[11]*2.78*cos(theta[2] - theta[11]) - v[2]*v[11]*-17.2*sin(theta[2] - theta[11])
Edge (3, 2) - Active Power Flow: v[3]**2*2.41 - v[3]*v[2]*2.41*cos(theta[3] - theta[2]) - v[3]*v[2]*-16.8*sin(theta[3] - theta[2])
Edge (3, 4) - Active Power Flow: v[3]**2*1.98 - v[3]*v[4]*1.98*cos(theta[3] - theta[4]) - v[3]*v[4]*-11.7*sin(theta[3] - theta[4])
Edge (3, 9) - Active Power Flow: v[3]**2*3.23 - v[3]*v[9]*3.23*cos(theta[

### **Amount of reactive power flow (q_var) between nodes**

$$\begin{aligned}
q_{kℓ} = -(v_k)^2 * b_{kℓ} + v_k * v_ℓ * b_{kℓ} * cos(θ_k − θ_ℓ) − v_k * v_ℓ * g_{kℓ} * sin(θ_k − θ_ℓ)
\end{aligned}$$

In [18]:
def reactive_power_flow_rule(model, k, l):
    if (k, l) in model.bkl:
        q_flow = - (model.v[k] ** 2) * model.bkl[k, l] + model.v[k] * model.v[l] * model.bkl[k, l] * pyo.cos(model.theta[k] - model.theta[l]) - model.v[k] * model.v[l] * model.gkl[k, l] * pyo.sin(model.theta[k] - model.theta[l])
        print(f"Edge ({k}, {l}) - Reactive Power Flow: {q_flow}")
        return model.q_var[k, l] == q_flow
    else:
        return pyo.Constraint.Skip

model.reactive_power_flow_constraint = pyo.Constraint(model.NODE, model.NODE, rule=reactive_power_flow_rule)

Edge (1, 2) - Reactive Power Flow: - v[1]**2*-20.1 + v[1]*v[2]*-20.1*cos(theta[1] - theta[2]) - v[1]*v[2]*4.12*sin(theta[1] - theta[2])
Edge (1, 11) - Reactive Power Flow: - v[1]**2*-22.3 + v[1]*v[11]*-22.3*cos(theta[1] - theta[11]) - v[1]*v[11]*5.67*sin(theta[1] - theta[11])
Edge (2, 1) - Reactive Power Flow: - v[2]**2*-20.1 + v[2]*v[1]*-20.1*cos(theta[2] - theta[1]) - v[2]*v[1]*4.12*sin(theta[2] - theta[1])
Edge (2, 3) - Reactive Power Flow: - v[2]**2*-16.8 + v[2]*v[3]*-16.8*cos(theta[2] - theta[3]) - v[2]*v[3]*2.41*sin(theta[2] - theta[3])
Edge (2, 11) - Reactive Power Flow: - v[2]**2*-17.2 + v[2]*v[11]*-17.2*cos(theta[2] - theta[11]) - v[2]*v[11]*2.78*sin(theta[2] - theta[11])
Edge (3, 2) - Reactive Power Flow: - v[3]**2*-16.8 + v[3]*v[2]*-16.8*cos(theta[3] - theta[2]) - v[3]*v[2]*2.41*sin(theta[3] - theta[2])
Edge (3, 4) - Reactive Power Flow: - v[3]**2*-11.7 + v[3]*v[4]*-11.7*cos(theta[3] - theta[4]) - v[3]*v[4]*1.98*sin(theta[3] - theta[4])
Edge (3, 9) - Reactive Power Flow: - v

### **Power Balance: generation and consumption (at each node)**

$$\begin{aligned}
\text{PowerGeneration}=\text{PowerConsumption....(at each node)}  
\end{aligned}$$

In [19]:
generator= 0
print(model.max_capacity[generator])
for generator in model.max_capacity: print(generator)

0.02
0
1
2
3
4
5
6
7
8


In [20]:
def active_power_balance_rule(model, k):
    # total active power injection at node k
    total_active_power_injection = (
        sum(model.p_var[k, l] for l in node_to_edges[k]) 
        - sum(model.p_var[l, k] for l in node_to_edges[k]) 
        + sum(model.max_capacity[g] * model.active_power[g] for g in model.GENERATOR if model.generator_location[g] == k)
    )

    total_active_power_demand = sum(model.demand[c] for c in model.CONSUMER if model.consumer_location[c] == k)
    print(f"Node {k}: Active Power Injection: {total_active_power_injection}, Active Power Demand: {total_active_power_demand}")
    return total_active_power_injection == total_active_power_demand

model.active_power_balance_constraint = pyo.Constraint([k for k in model.NODE], rule=active_power_balance_rule)

Node 1: Active Power Injection: p_var[1,2] + p_var[1,11] - (p_var[2,1] + p_var[11,1]), Active Power Demand: 0.1
Node 2: Active Power Injection: p_var[2,1] + p_var[2,3] + p_var[2,11] - (p_var[1,2] + p_var[3,2] + p_var[11,2]) + 0.02*active_power[0] + 0.15*active_power[1] + 0.08*active_power[2], Active Power Demand: 0
Node 3: Active Power Injection: p_var[3,2] + p_var[3,4] + p_var[3,9] - (p_var[2,3] + p_var[4,3] + p_var[9,3]) + 0.07*active_power[3], Active Power Demand: 0
Node 4: Active Power Injection: p_var[4,3] + p_var[4,5] - (p_var[3,4] + p_var[5,4]) + 0.04*active_power[4], Active Power Demand: 0.19
Node 5: Active Power Injection: p_var[5,4] + p_var[5,6] + p_var[5,8] - (p_var[4,5] + p_var[6,5] + p_var[8,5]) + 0.17*active_power[5], Active Power Demand: 0
Node 6: Active Power Injection: p_var[6,5] + p_var[6,7] - (p_var[5,6] + p_var[7,6]), Active Power Demand: 0.11
Node 7: Active Power Injection: p_var[7,6] + p_var[7,8] + p_var[7,9] - (p_var[6,7] + p_var[8,7] + p_var[9,7]) + 0.17*active_

In [21]:
def reactive_power_balance_rule(model, k):
    # total reactive power injection at node k
    total_reactive_power_injection = (
        sum(model.q_var[k, l] for l in node_to_edges[k])
        - sum(model.q_var[l, k] for l in node_to_edges[k])
        + sum((model.v[k] ** 2) * (model.bkl[k, l] - model.gkl[k, l] * pyo.cos(model.theta[k] - model.theta[l]))
            for l in node_to_edges[k])
    )
    print(f"Node {k} - Reactive Power Injection: {total_reactive_power_injection}")
    return total_reactive_power_injection == 0.0

model.reactive_power_balance_constraint = pyo.Constraint([k for k in model.NODE], rule=reactive_power_balance_rule)

Node 1 - Reactive Power Injection: q_var[1,2] + q_var[1,11] - (q_var[2,1] + q_var[11,1]) + v[1]**2*(-20.1 - 4.12*cos(theta[1] - theta[2])) + v[1]**2*(-22.3 - 5.67*cos(theta[1] - theta[11]))
Node 2 - Reactive Power Injection: q_var[2,1] + q_var[2,3] + q_var[2,11] - (q_var[1,2] + q_var[3,2] + q_var[11,2]) + v[2]**2*(-20.1 - 4.12*cos(theta[2] - theta[1])) + v[2]**2*(-16.8 - 2.41*cos(theta[2] - theta[3])) + v[2]**2*(-17.2 - 2.78*cos(theta[2] - theta[11]))
Node 3 - Reactive Power Injection: q_var[3,2] + q_var[3,4] + q_var[3,9] - (q_var[2,3] + q_var[4,3] + q_var[9,3]) + v[3]**2*(-16.8 - 2.41*cos(theta[3] - theta[2])) + v[3]**2*(-11.7 - 1.98*cos(theta[3] - theta[4])) + v[3]**2*(-19.4 - 3.23*cos(theta[3] - theta[9]))
Node 4 - Reactive Power Injection: q_var[4,3] + q_var[4,5] - (q_var[3,4] + q_var[5,4]) + v[4]**2*(-11.7 - 1.98*cos(theta[4] - theta[3])) + v[4]**2*(-10.8 - 1.59*cos(theta[4] - theta[5]))
Node 5 - Reactive Power Injection: q_var[5,4] + q_var[5,6] + q_var[5,8] - (q_var[4,5] + q_var[

## Set Objective Function

**To minimize the cost of total active power generated for consumption:**

$$\begin{aligned}
Minimize\ (Cost_{activePower})
\end{aligned}$$


In [22]:
def objective_rule(model):
    objective_value = sum(
        model.cost[g] * model.active_power[g]
        for g in model.GENERATOR
    )
    return objective_value

model.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

In [23]:
solver = SolverFactory('ipopt', executable='C:\\ipopt\\bin\\ipopt.exe')
solver.solve(model, tee='solver_output.txt')

Ipopt 3.11.1: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

NOTE: You are using Ipopt by default with the MUMPS linear solver.
      Other linear solvers might be more efficient (see Ipopt documentation).


This is Ipopt version 3.11.1, running with linear solver mumps.

Number of nonzeros in equality constraint Jacobian...:      503
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       93

Exception of type: TOO_FEW_DOF in file "../../../../Ipopt/src/Interfaces/IpIpoptApplication.cpp" at line 860:
 Exception message: status != TOO_FEW_DEGREES_OF_FREEDOM evaluated false: Too few 

{'Solver': [{'Status': 'warning', 'Message': 'Too few degrees of freedom (rethrown)!', 'Termination condition': 'other'}]}

## Results

In [ ]:
active_power = {k: pyo.value(model.active_power[k]) for i in model.GENERATOR}
reactive_power = {k: pyo.value(model.reactive_power[k]) for i in model.GENERATOR}
voltage_amplitudes = {k: pyo.value(model.v[k]) for i in model.NODE}
voltage_angles = {k: pyo.value(model.theta[k]) for i in model.NODE}
total_cost = pyo.value(model.objective)

In [ ]:
print("Active Power Production:", active_power)
print("Reactive Power Production:", reactive_power)
print("Voltage Amplitudes:", voltage_amplitudes)
print("Voltage Angles:", voltage_angles)
print("Total Cost of Production:", total_cost)

In [ ]:
model.pprint()